# Embed a holoviews interactive chart in a folium map (0.0.1-alpha)

The out out of this notebook can be seen at

https://sfsheath.github.io/holoviews-in-folium/embedded_iframe.html

All very "quick and dirty". Is the an approach that doesn't require writing holoviews variable to a file? I tried using a hv.Renderer object, but the 'raw' html that generates has dependencies that are met when it's written out to an html file.

In [ ]:
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import holoviews as hv
hv.extension('bokeh')
# hv.extension('mapplotlib')

from bokeh.models import HoverTool
from folium import FeatureGroup, LayerControl, Map, Marker



from IPython.display import HTML,IFrame

%matplotlib inline

## Load Data

In [ ]:
ramphs_df = pd.read_csv('http://sebastianheath.com/roman-amphitheaters/roman-amphitheaters.csv')


In [ ]:
# set up df for an index plot
extmajors_df = ramphs_df.sort_values(by = 'extmajor').reset_index()[['index','label','extmajor','latitude','longitude']]

extmajors_df['index'] = extmajors_df.index

extmajors_df.index = extmajors_df.label

extmajors_df.head(2)

## Make the Holoviews Viz

In [ ]:
use_label = 'Tours' # choose which amphitheater to highlight

In [ ]:
hover = HoverTool(tooltips=[("", "@label")])

index_plot = hv.Points(extmajors_df[['index', 'extmajor','label']].dropna())\
 .opts(plot=dict(height=300, width=400, tools=[hover],color=hv.Cycle('Blues')))

h_line = hv.HLine(extmajors_df.loc[use_label,'extmajor']).options(color = 'red', line_width= 1)

embed_in_folium = (index_plot * h_line).relabel("Exterior lengths with {} highlighted".format(use_label))

In [ ]:
%%output filename="embed_in_folium" fig="html" 
# I'd like to skip this step. Can I?? Will keep working on it.

embed_in_folium

## Make Folium map
The content of the iframe is the holoviews viz saved to html.

In [ ]:
# get the lat/long of the amphitheater
latitude = extmajors_df.loc[use_label,'latitude']
longitude = extmajors_df.loc[use_label,'longitude']

m = folium.Map([latitude, longitude], zoom_start=4)

# get the holoviews out as a variable
with open('embed_in_folium.html', 'r') as myfile:
    html = myfile.read()

# create an iframe, then a popup with that iframe as content
iframe = folium.IFrame(html=html, width=450, height=300)
popup = folium.Popup(iframe, max_width=2650)

folium.CircleMarker([latitude, longitude], radius=5,popup=popup).add_to(m)

m.save("embedded_iframe.html", )
m
